pytorch安装

In [3]:
import torch

In [4]:
torch.cuda.is_available()

True

查询package里的函数以及函数如何使用

In [11]:
dir(torch.cuda)

['Any',
 'BFloat16Storage',
 'BFloat16Tensor',
 'BoolStorage',
 'BoolTensor',
 'ByteStorage',
 'ByteTensor',
 'CUDAGraph',
 'CUDAPluggableAllocator',
 'Callable',
 'CharStorage',
 'CharTensor',
 'ComplexDoubleStorage',
 'ComplexFloatStorage',
 'CudaError',
 'DeferredCudaCallError',
 'Device',
 'DoubleStorage',
 'DoubleTensor',
 'Event',
 'ExternalStream',
 'FloatStorage',
 'FloatTensor',
 'HalfStorage',
 'HalfTensor',
 'IntStorage',
 'IntTensor',
 'List',
 'LongStorage',
 'LongTensor',
 'MemPool',
 'MemPoolContext',
 'Optional',
 'OutOfMemoryError',
 'ShortStorage',
 'ShortTensor',
 'Stream',
 'StreamContext',
 'Tuple',
 'Union',
 '_CudaBase',
 '_CudaDeviceProperties',
 '_DeviceGuard',
 '_HAS_PYNVML',
 '_LazySeedTracker',
 '_PYNVML_ERR',
 '_WrappedTritonKernel',
 '__all__',
 '__annotations__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_cached_device_count',
 '_check_bf16_tensor_supported',
 '_check_c

In [12]:
#查函数不要加括号
help(torch.cuda.is_available)

Help on function is_available in module torch.cuda:

is_available() -> bool
    Return a bool indicating if CUDA is currently available.



**读取数据:dataset与dataloader**

dataset:提供一种方式获取数据及其label

dataloader：为后面的网络提供不同的数据形式

In [15]:
from torch.utils.data import Dataset
from PIL import Image
import os

In [ ]:
#自定义Dataset类用于读取图片数据
class MyDataset(Dataset):
    def __init__(self,root_dir,label_dir):
        self.root_dir = root_dir #数据集根目录
        self.label_dir = label_dir #标签目录 ants or bees
        self.path = os.path.join(self.root_dir,self.label_dir) #拼接路径
        self.img_path = os.listdir(self.path) #读取路径下的所有文件名称
    
    def __getitem__(self,idx):
        img_name = self.img_path[idx] #获取图片名称
        img_path = os.path.join(self.path,img_name) #拼接图片路径
        img = Image.open(img_path) #读取图片
        label = self.label_dir
        return img,label
    def __len__(self):
        return len(self.img_path) 

In [ ]:
#读取蚂蚁数据集
root_dir = r'F:\RUC\pytorch\数据集\hymenoptera_data\train'
label_dir = 'ants'
ants_dataset = MyDataset(root_dir,label_dir)

In [25]:
#打开图片
img,label = ants_dataset.__getitem__(0)
#或者使用:
#img,label = ants_dataset[0]
img.show()

#查看数据集大小
ants_dataset.__len__()
#或者使用:
#len(ants_dataset)

124

In [26]:
root_dir = r'F:\RUC\pytorch\数据集\hymenoptera_data\train'
label_dir = 'bees'
bees_dataset = MyDataset(root_dir,label_dir)

In [27]:
#合并数据集
train_dataset = ants_dataset + bees_dataset

In [28]:
#查看合并数数据集的信息
img,label = train_dataset.__getitem__(125)
img.show()

In [ ]:
#如果是图片+标签的形式 如何读取数据
class MyDataset2(Dataset):
    def __init__(self,root_dir,image_dir,label_dir):
        self.root_dir = root_dir #数据集根目录
        self.image_dir = image_dir #图片目录
        self.label_dir = label_dir #标签目录
        self.image_path = os.path.join(self.root_dir,self.image_dir) #拼接路径并读取
        self.label_path = os.path.join(self.root_dir,self.label_dir)
        self.image_list = os.listdir(self.image_path)
        self.label_list = os.listdir(self.label_path)
        # 因为label 和 Image文件名相同，进行一样的排序，可以保证取出的数据和label是一一对应的
        self.image_list.sort()
        self.label_list.sort()
 
    def __getitem__(self,idx):
        #图片、标签名称/路径
        img_name = self.image_list[idx]
        label_name = self.label_list[idx]
        img_item_path = os.path.join(self.root_dir, self.image_dir, img_name)
        label_item_path = os.path.join(self.root_dir, self.label_dir, label_name)
        #读取图片、标签
        img = Image.open(img_item_path)

        with open(label_item_path, 'r') as f:
            label = f.readline()

        # img = np.array(img)
        #?这里为什么要转换为tensor
        #img = self.transform(img) 暂时先不用transform
        sample = {'img': img, 'label': label}
        return sample

    def __len__(self):
        return len(self.label_list) 

In [39]:
root_dir = r'F:\RUC\pytorch\数据集\练手数据集\train'
image_dir = 'ants_image'
label_dir = 'ants_label'

ants_dataset = MyDataset2(root_dir,image_dir,label_dir)

In [42]:
#打开图片
ants_dataset[0]['img'].show()